# K-means clustering

For this weeks assignment, you will implement a *clustering* algorithm called K-means. Up to this point, all the algorithms we've covered have been *supervised learning*, but clustering is a form of *unsupervised learning*. In unsupervised learning, there is no target variable we are trying to approximate. Instead, we are trying to extract some kind of pattern from the data, where the type of pattern depends on the algorithm we use. In the case of clustering, we are looking for *clusters* or groupings within the data. The exact shape of these clusters of course depends on how we define the distance between each of the data points, which is what the second half of this assignment is all about.

We will work with a classic dataset from machine learning; Fisher's Iris dataset. The dataset contains the measurements of *length* and *width* of the *sepals* and *petals* of 150 flowers. 

<img src="https://upload.wikimedia.org/wikipedia/commons/7/78/Petal-sepal.jpg" width=250>

The dataset contains 4 attributes for each flower; *length* and *width* of both *sepals* and *petals*. In addition, this data set also actually contains the supervised label of the species of Iris, but we will completely **ignore that part of the data set for this assignment**. So the assignment we're trying to solve is to find out how these 4 for properties are distributed for all these 150 flowers, and if we can find any groupings of points that are sufficiently similar (for some definition of similarity) that we might consider them to be their own species.

The Iris dataset is such a classic example that it even comes included in `seaborn`, and so can easily be loaded with the code below:

In [ ]:
%matplotlib inline

import math
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns

# Load the data set
df_iris = sns.load_dataset('iris')

# Remove the target variable
df_iris = df_iris.drop('species', axis=1)

print(df_iris)

## Exploration

Lets plot this data, so we can see what it looks like. Below is some seaborn code that plots a so-called scatter matrix. This is a grid of scatter plots that can be used to easily identify correlations and see the seperability of the dataset. The diagonals of this plot show how the data is distributed for every *single* feature, while the non-diagonals show scatter plots using *two* of the features as the $x$ and $y$ axis, with all the different possible configurations shown in the grid.

In [ ]:
sns.pairplot(df_iris)

plt.show()

This type of graphic is a nice way to visually explore what the data set looks like, and to determine what features might be useful. Normally, if you were to solve this clustering problem, you would probably just use all 4 features to determine the clusters. However, the goal of this assignment is to understand clustering algorithms, so having clear visuals will be essential. For this reason, we'll select 2 features to use, so we can make nice 2 dimensional plots containing all the features. From the plot above, it seems like `petal_length` and `petal_width` are the most distinctive features, so we'll use those.

**Note:** Although we will only be working with 2 features, all functions and code for this assignment should work with *any* number of features.

In [ ]:
data = df_iris.drop(['sepal_length', 'sepal_width'], axis=1).to_numpy()

print(data)

## One-hot encoding

Before we get started with k-means, we'll briefly introduce a bit of data representation convention, called *one-hot encoding*, that will simplify the equations and implemenation later in the assignment. Any categorical feature, so features consisting of a distinct and finitely limited set of options, can be represented with a one-hot encoding. Here, we will use one-hot encoding to represent the value $c^{i}$ from the theory videos, so the index of the cluster centroid to which example $x^{i}$ is currently assigned, which is an integer in the range $1$ to $k$.

As an example, let's say we have $m=5$ data points and $k=3$ clusters. At some point in the cluster algorithm, we have the following vector for $c$:

$$c = \left[\begin{array}{c} 2 \\ 1 \\ 1 \\ 3 \\ 2 \end{array} \right]$$

We could represent this exact same information with the following one-hot encoded matrix $B$:

$$ B = \left[\begin{array}{ccc}
0 & 1 & 0\\
1 & 0 & 0\\ 
1 & 0 & 0\\
0 & 0 & 1\\ 
0 & 1 & 0\\
\end{array} \right]$$

In this matrix $B$, the rows $b^i$ still correspond to each sample $x^i$, but the columns now correspond to each of the different clusters. All the values for a row are $0$, except for the column corresponding to the cluster centroid to which that point $x^i$ is assigned, which will be $1$. So there is always exactly one value in a row that is *on* (or *hot*), and all the other are *off* (or *cold*), which is why it is called a *one-hot encoding*.

There are quite a few places in machine learning where a one-hot encoding ends up working better than just a categorical feature, but we won't go into all of those now. For now, it will just make the equations and implemenation of *k-means* a little easier.

As short seperate exercise, implement the function `categorical_to_one_hot` below, that transforms a vector of categorical values into a one-hot encoded matrix. You won't need this function for the rest of the assignment, but will be good practice.

*Hint:* First, use [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) to determine the set of categorical values and thus the size of the matrix. Then, build a matrix of correct size using [np.zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) and add ones where needed.

In [ ]:
def categorical_to_one_hot(cat_vec):
    ### YOUR SOLUTION HERE
    
    
print(categorical_to_one_hot([2, 1, 1, 3, 2]))

# K-means 

A slightly modified version of the pseudo-code from the theory videos, using the one-hot encoding, can be found below:

* Randomly initialize all $\mu_j$, with $j = 1, \dots, k$
* Until $\mu$ converges:
    * For all $x^i$, with $i = 1, \dots, m$
        * $b^i_j \leftarrow \left\{\begin{array}{ll} 1 & if\ dist(x^i,  \mu_j) \ =\ min_d\ dist(x^i, \mu_d)\\ 
            0 & otherwise \\ \end{array}\right.$
    * For all $\mu_j$, with $j = 1, \dots, k$
        * $\mu_j \leftarrow \sum_i b^i_j x^i / \sum_i b^i_j$

As this pseudo-code is a little dense, let's unpack all of the steps first:

1. Initialize the cluster means
2. While not converged: \
    2.1 Compute the new cluster assignment \
    2.2 Compute the new cluster means 
    
#### 1. Initialize the cluster means

Initializing the cluster means creates a starting point for the algorithm. There are several ways you might create random starting points, but you usually select $k$ random points from $X$, which ensures that each of the means have sensible starting values for the data set you're working with.

#### 2. While not converged

This simply means the steps of k-means are repeated until the cluster means no longer improve. It is actually possible to prove that this must occur at some point, i.e. that there can't be *loops* in the assignment or anything like that. You could use a cost function to detect that the centroids are no longer improving, but the most direct method is simply to compare the previous matrix $\mu$ with the current one, and see there was litte or no change.

#### 2.1 Compute the new cluster assignment

This creates one-hot encoding matrix $B$ containing the cluster assignments, as describe in the section before, but here in a mathematical notation:

$$b^i_j \leftarrow \left\{\begin{array}{ll} 1 & if\ dist(x^i,  \mu_j) \ =\ min_d\ dist(x^i, \mu_d)\\ 
            0 & otherwise \\ \end{array}\right.$$

$b^i$ is the one-hot encoded vector for the cluster assignment of point $x^i$, with the active column corresponding to the index of closest cluster mean.

#### 2.2 Compute the new cluster means

This creates the matrix $\mu$ containing all the cluster means, recomputed for the newly assigned samples:

$$\mu_j \leftarrow \frac{\sum_i b^i_j x^i}{\sum_i b^i_j}$$

Because of the one-hot encoded matrix $B$ this equation became a little simpler. As $x^i$ is a vector of features for a single sample, the mean $\mu_j$ must also be a vector, containing the means for each of the features, for that specific $j^{th}$ cluster. Summing together all vectors, gives a vector of totals for each feature. By multiplying with $b^i_j$, we filter out the samples that are not part of the $j^{th}$ cluster, as these will get multiplied by $0$. Summing together $b^i_j$ for a specific $j$, ends up just counting how many pointer there are in that cluster. So all together, the equation above computes the mean vector for the cluster with index $j$.

## Initializing the clusters

This is the first function we'll write for k-means. The function should randomly initialize `k` means, by selecting `k` random points from the `data`. This function should work with data of any dimensionality, e.g. not depend on there being specifically 2 or 4 features in the data set.

In [ ]:
def init_clusters(data, k):
    ### YOUR SOLUTION HERE
    


In [ ]:
assert init_clusters(data, 1).shape == (1,2)
assert init_clusters(data, 4).shape == (4,2)
assert init_clusters(np.array([[1,2,3], [3, 2, 1]]), 2).shape == (2, 3)

## Computing the distance

This function should compute the *Euclidean* distance between 2 $n$-dimensional vectors `p` and `q`, which is defined as:

$$ dist(p,q) = \sqrt{\sum^n_{i=1}(p_i - q_i)^2} $$

In [ ]:
def eucl_dist(p, q):
    ### YOUR SOLUTION HERE
    


In [ ]:
assert eucl_dist(np.array([3, 3]), np.array([1, 3])) == 2
assert eucl_dist(np.array([4, 2]), np.array([5, 1])) == math.sqrt(2)
assert eucl_dist(np.array([1, 2, 3]), np.array([3, 2, 1])) == math.sqrt(8)

## Cluster assignments

This function should compute the one-hot encoded matrix $B$ containing the assignments of `data` points to their nearest clusters, based on the current cluster `means` and a `dist_func` to determine *nearness*. The `dist_func` argument should be a *function* that computes some distance metric between points. For now, you can assume this is always a *Euclidean* distance, so the function `eucl_dist` defined in the cell above, but later in the assignment we will use other distance metrics here too.

You can use an argument function in exactly the same way you use a normal function, so `dist_func(p, q)` will compute the distance between `p` and `q`, whatever distance function is used. You can take some inspiration from `categorical_to_one_hot` for constructing your $B$ matrix.

In [ ]:
def cluster_assignment(data, means, dist_func):
    ### YOUR SOLUTION HERE
    


In [ ]:
means = np.array([[0, 0], [999999, 999999]])

assert cluster_assignment(data, means, eucl_dist).shape == (len(data), len(means)), "The output has the incorrect shape. It should have shape (len(data), len(means))."
assert np.sum(cluster_assignment(data, means, eucl_dist), axis=0)[0] == 150, "Not everything was assigned to cluster 0."
assert np.allclose(np.ones((len(data), 1)), np.sum(cluster_assignment(data, means, eucl_dist), axis=1)), "There are rows with more than or less than one \"1\""

means = np.array([[4.2, 1.3], [5.6, 2.2], [1.7, 0.2], [5.,  1.5], [4.6, 1.4]])

assert cluster_assignment(data, means, eucl_dist).shape == (len(data), len(means)), "The output has the incorrect shape. It should have shape (len(data), len(means))."
assert np.sum(cluster_assignment(data, means, eucl_dist)) == 150, "Not everything was assigned to a cluster."
assert np.allclose(np.ones((len(data), 1)), np.sum(cluster_assignment(data, means, eucl_dist), axis=1)), "There are rows with more than or less than one \"1\""
assert np.allclose(cluster_assignment(data, means, eucl_dist), np.array([[0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,1.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,1.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,1.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.]])), "Points were not properly assigned to the closest cluster mean."

## Compute the means

This function should compute the matrix $\mu$ containing the new mean vectors, based on the current assignment of `data` points to clusters, as described by the one-hot encoded matrix `b`. Each mean vector should contain the mean for the $n$ features for that cluster, and there should be $k$ mean vectors in total, so the whole matrix $\mu$ should be $k \times n$.

**Note:** It is actually possible for a cluster to get *no assigned points*. If this is the case, then it is impossible to calculate a new mean, so you should instead use a new random point from `data` as that cluster mean. This way the number of clusters will always remain a constant $k$ and at the next iteration the algorithm will assign at least one point to this cluster (i.e. the point you selected).

In [ ]:
def compute_means(data, b):
    ### YOUR SOLUTION HERE
    


In [ ]:
assert compute_means(np.array([[1, 3, 0], [0, 2, 3], [1, 0, 1]]), np.array([[1, 0],[1, 0], [0, 1]])).shape == (2, 3), "The output shape of your function is incorrect."
assert np.allclose(compute_means(np.array([[1, 3, 0], [0, 2, 3], [1, 0, 1]]), np.array([[1, 0],[1, 0], [0, 1]])), np.array([[0.5, 2.5, 1.5], [1, 0, 1]])), "The computed means are incorrect."

b = np.array([[0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,1.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,1.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,1.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,1.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,0.,0.,0.,1.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [1.,0.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,0.,1.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.], [0.,1.,0.,0.,0.], [0.,1.,0.,0.,0.], [0.,0.,0.,1.,0.]])

assert np.allclose(compute_means(data, b), np.array([[3.92222222, 1.1962963], [5.79142857, 2.12857143], [1.462, 0.246], [5.00526316, 1.74736842], [4.57368421, 1.45263158]])), "The computed means are incorrect."

assert compute_means(np.array([[1, 3, 0], [0, 2, 3], [1, 0, 1]]), np.array([[1, 0],[1, 0], [1, 0]]))[1] in  np.array([[1, 3, 0], [0, 2, 3], [1, 0, 1]]), "The compute_means function does not properly handle clusters that have no assigned points."

## Checking convergence

This function should determine if the algorithm has converged based on the values of the `old` and `new` cluster means after one step of the algorithm. If the `old` and the `new` means are close enough to consider them unchanged, the function should return `True` and otherwise it should return `False`.


In [ ]:
def is_converged(old, new):
    ### YOUR SOLUTION HERE
    


In [ ]:
assert is_converged(np.array([[1, 0]]), np.array([[1, 0]])), "Incorrect output for two equal mean matrices."
assert is_converged(np.array([[1, 0, 0], [0, 1, 0]]), np.array([[1, 0, 0], [0, 1, 0]])), "Incorrect output for two equal mean matrices."
assert is_converged(means, means), "Incorrect output for two equal mean matrices."
assert not is_converged(np.array([[1, 0]]), np.array([[0, 1]])), "Incorrect output for two unequal mean matrices."

## Plotting the clusters

This function should plot the `means` and their assigned `data` points as describe by `b`. Each cluster mean should be a cross and its assigned points should be dots, all be the same color, but each seperate cluster should be a represented by a seperate color.

There is a list of valid matplotlib `colors` in the code already, which you can use to distinguish each of the different clusters. Note that this plotting function can therefore only draw clusters with a maximum $k=8$. Each data point should also be assumed to be 2 dimensional, so $n=2$, in order to make the plotting feasible. This is the only function is this assignment where you should make these assumptions though.


In [ ]:
def plot_clusters(data, b, means):
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    ### YOUR SOLUTION HERE
    

plot_clusters(data, b, means)

## K-means

Combine all these functions into a general `k`-means function for some `data` set, as described by the pseudo-code below:

* Randomly initialize all $\mu_j$, with $j = 1, \dots, k$
* Until $\mu$ converges:
    * For all $x^i$, with $i = 1, \dots, m$
        * $b^i_j \leftarrow \left\{\begin{array}{ll} 1 & if\ dist(x^i,  \mu_j) \ =\ min_d\ dist(x^i, \mu_d)\\ 
            0 & otherwise \\ \end{array}\right.$
    * For all $\mu_j$, with $j = 1, \dots, k$
        * $\mu_j \leftarrow \sum_i b^i_j x^i / \sum_i b^i_j$

The closest means for the clusters assignment step should depend on exactly what `dist_func` is used to define closeness. When converged, the `kmeans` function should return the matrix of convered means `mu`.


In [ ]:
def kmeans(data, k, dist_func):
    ### YOUR SOLUTION HERE
    


In [ ]:
assert kmeans(data, 3, eucl_dist).shape == (3, data.shape[1]), "Your output has the incorrect shape."
assert kmeans(data, 12, eucl_dist).shape == (12, data.shape[1]), "Your output has the incorrect shape."
assert np.all([mean in np.array([[1, 3, 0], [0, 2, 3], [1, 0, 1]]) for mean in kmeans(np.array([[1, 3, 0], [0, 2, 3], [1, 0, 1]]), 3, eucl_dist)]), "Your function assigns incorrect means"

## Inspecting the steps of k-means

For this next part of the assignment, we're going to take a look a the steps of the k-means algorithm. To start with, copy your code from `kmeans` into the cell below, but without the function-level indent. The code should use a fixed value of $k=3$ and the `dist_func` should be the *Euclidean* distance. The data set will of course be the Iris data set.

In order to be able to inspect the intermediate results of the k-means algorithm, you will need to plot the clusters and assigned points at different staged of the k-means algorithm, *before* it has convered. You should not plot *all* the steps though, as the will be too many. I think the most informative plots are the first 3 steps of k-means and the final converged result, but you may select your own intermediate steps. Modify your code to plot 4 specific intermediate steps of *k-means* and label each of the plots accordingly.

Check these steps all look correct before continuing the assignment, as this is the easiest visual way to check if k-means is actually doing the correct steps each iteration.

In [ ]:
### YOUR SOLUTION HERE


# Distance metrics

If everything in the previous cells works correctly, you now have a complete implementation of the k-means algorithm. When computing the means closest to each point, the instruction was to explicitly leave the distance function *variable*, as a function argument. However, Euclidean distance, or the *straight-line* distance, is by far the most common and intuitive distance measure, so you might have been wondering what the alternatives even are. The next part of this assignment is about answering exactly that question.

To start with, we're going try and think a little more *visually* about how distance can be represented and what effect this has on the shape of the possible clusters. Below is some code to create a plot of k-means clusters using your functions. In addition to the means and the assigned points, this function also plots a series of contours, indicating the shape of the cluster for different fixed distances using the provided distance function.

#### What cluster shape do you think using Euclidean distance to determine the nearest mean will result in? Explain your anwer.

*Your answer goes here.*

**Note:** Your answer here, and for the following questions, will not be considered correct or incorrect, but only evaluated on if you attemped to give a well reasoned explanation for the shape. Running the code below will always reveal the actual answer of course.


In [ ]:
def cluster_contour_plot(data, k, dist_func, title, max_dist):
    # Constants for the plot
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    samps, mr = 250, 3.2
    
    # Create the figure and scale the axes
    plt.figure(figsize=(16,16))
    plt.xlim((-mr, mr))
    plt.ylim((-mr, mr))
    plt.title(title, fontsize=32)

    # Create the grid
    X, Y = np.meshgrid(np.linspace(-mr, mr, samps), np.linspace(-mr, mr, samps))

    # Compute the clusters and their assignments
    means = kmeans(data, k, dist_func)
    b = cluster_assignment(data, means, dist_func)
    
    # Plot the distance contours for each cluster
    for i in range(means.shape[0]):
        pdf_map = np.vectorize(lambda x,y: dist_func(np.array([x, y]), means[i, :]))
        Z = pdf_map(np.ravel(X), np.ravel(Y)).reshape(X.shape)

        plt.contour(X, Y, Z, levels=np.linspace(0, max_dist, 5), colors=colors[i], alpha=0.35)

    # Plot the clusters and points
    plot_clusters(data, b, means)

# Center the data around the origin
centered_data = data - np.mean(data, axis=0)

# Create the plot
cluster_contour_plot(centered_data, 3, eucl_dist, "Euclidean distance", 0.75)


### Inspecting the results

Some important things to note about the plot above:

1. The clusters are all shaped like circles. This is because all the points that are the same straight line distance away from the mean will together form a circle.
2. The plot is based on the `kmeans` function, which uses the function `init_clusters`, which randomly selects starting means. Therefore, rerunning this plot might result in different means.
3. The data was shifted to be centered around the origin. This has no effect on the Euclidean distance, but will improve the results for some of the other distance metrics.
4. The plot now shows exactly the same scale for the x-axis and y-axis, resulting in a lot of empty space in the plot. If the plot was not scaled this way, so with the x-axis and y-axis scaled differently, it would be very hard to see that resulting shapes are actually circles.


## Manhattan distance

The Manhattan distance is also known as the [taxicab distance](https://en.wikipedia.org/wiki/Taxicab_geometry) or the city block distance, and is defined as:

$$ dist(p,q) = \sum^n_{i=1}|p_i - q_i| $$

Implement the function `man_dist`, which should calculate the Manhattan distance between two *Numpy* arrays `p` and `q`. The bar notation $|x|$ is the absolute value of $x$, and you can use [np.absolute](https://numpy.org/doc/stable/reference/generated/numpy.absolute.html) to compute this for an entire array. Your solution should not use any loops to compute the distance.

The Manhattan distance sums the absolute differences between the vectors $p$ and $q$ along every axis, so there are no "diagonal shortcuts" using this distance measure. This means that the resulting shape of these clusters cannot be a circle.

#### What cluster shape do you think using Manhattan distance to determine the nearest mean will result in? Explain your anwer.

*Your answer goes here.*


In [ ]:
def man_dist(p, q):
    ### YOUR SOLUTION HERE
    

cluster_contour_plot(centered_data, 3, man_dist, "Manhattan distance", 1)

## Angular distance

Cosine similarity is a metric based on the angle between two vectors, as seen from the *origin* (which is why we centered the data a couple of cells back). The theory for this was covered in the *SOWISO* module of this week. As stated there, the cosine of the angle $\phi$ between two vectors $p$ and $q$, can be computed as:

$$cos(\phi) = \frac{p \cdot q}{\|p\| \|q\|}$$

This definition always gives the shortest angle between $p$ and $q$, so $\phi$ will always be between $0$ and $\pi$ (with the angle in [radians](https://en.wikipedia.org/wiki/Radian)). Looking at a graph for the [cosine function](https://www.wolframalpha.com/input/?i=cos(x)+from+0+to+2pi) can also be helpful to think about the possible values for the function. Note that since $\phi$ is between $0$ and $\pi$, we're really only concerned with the first half of that graph.

From that same graph, we can also conclude that $cos(\phi)$ must fall between $1$ and $-1$, with $1$ corresponding to an angle of $0$ and $-1$ corresponding to an angle of $\pi$. It would be useful to generally convert $cos(\phi)$ back to just $\phi$, which is exactly what the *arccos* function is for:

$$\phi = arccos(cos(\phi))$$

Now, using this *arccos*, we can define a proper angular distance:

$$ dist(p, q) = arccos(\frac{p \cdot q}{\|p\| \|q\|}) $$

This will compute the angle between the two vectors, as seen from the origin, in radians. The minimum distance any two vector can have is $0$ and the maximum distance any two vectors can have is $\pi$.

Implement the function `angle_dist`, which should calculate the angular distance between two *Numpy* arrays `p` and `q`. Use the function [math.acos](https://docs.python.org/3/library/math.html#math.acos) to compute the *arccos*. *Numpy* also has a separate module of functions useful for [linear algebra](https://numpy.org/doc/stable/reference/routines.linalg.html), any of which you can also use. Your solution should not use any loops to compute this distance.

*Hint:* It is impossible for the *arccos* be computed for any value $<-1$ or $> 1$. However, due to floating point inaccuracies, it *is* possible for your solution to result in such values for $cos(\phi)$. Your function should detect if this happens, and correct these values to exactly $1$ or $-1$ if necessary.

### Sidenote: When would you need an angular distance?

It might seem strange to use the angle as a distance metric between data points, but it is actually very common in certain parts of machine learning. The most common example is when comparing similarity between documents. Here you would define every document as a vector of word counts, so each feature is the count for one specific word. According to this metric, when comparing documents, you don't care if the actual counts of the individual words are close, but only if the distribution / ratio between the words is approximately the same. Using this metric, documents with same distribution of words, but containing twice as many of every word, would produce vectors in the same direction and so result in an angle close to 0.

#### What cluster shape do you think using angular distance to determine the nearest mean will result in? Explain your anwer.

*Your answer goes here.*


In [ ]:
def angle_dist(p, q):
    ### YOUR SOLUTION HERE
    

cluster_contour_plot(centered_data, 3, angle_dist, "Angular distance", 0.12)

## Mahalanobis distance

The Mahalanobis distance computes the distance between two points $p$ and $q$, scaled by the *variance* and *covariance* of their distribution. This means that if there is a larger variance for one feature (for example, the x-axis feature in the data set for this assignment) than for another (e.g. the y-axis feature), then the feature that "naturally varies more", will actually add *less* to the distance. Similarly, if two features are strongly correlated, then their combined increase/decrease will also add less to the distance, as this is the *expected shape* of the distribution of the data.

More formally, the Mahalanobis distance actually computes the number of standard deviations two points are apart, according to a multivariate Gaussian distribution defined by a covariance matrix $S$. The covariance matrix is just a matrix containing all the variances and covariances between all the features. If there are $n$ features in your data set, the covariance matrix for that data set will be $n \times n$, and all the variances will be along the diagonal of that matrix. The Mahalanobis distance is then defined as:

$$ dist(p, q) = \sqrt{(p - q)^TS^{-1}(p - q)} $$

Implement the function `maha_dist`, which should calculate the Mahalobis distance between two points `p` and `q`, assuming some covariance matrix `S`. The starting code below might seem a little strange, but it is actually correct and the best way to define a distance function that will work nicely with the rest of the code. Technically, the function `maha_dist_generator` creates a closure which computes the sample covariance matrix of the data and returns the `maha_dist` function with that specific sample covariance. Closures are a nice *Python* feature, but we won't go into detail here. For your solution of `maha_dist`, you should just assume `S` is already correctly defined and available in that function. Use [np.linalg.inv](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) to compute $S^{-1}$, the inverse of the sample covariance matrix.

The reason you might want to use a Mahalanobis distance is a little more intuitive than with an angular distance. If you have features which are of a complete different scale, then a *Euclidean* distance will always weight the feature with the larger differences much more heavily. In order to get a better "relative" closeness, you can scale all your features to be in the same range (also a common solution), or you can just use a distance function that actually computes a scaled distance.

#### What cluster shape do you think using Mahalanobis distance to determine the nearest mean will result in? Explain your anwer.

*Your answer goes here.*


In [ ]:
def maha_dist_generator(data):
    S = np.cov(data.T)
        
    def maha_dist(p, q):
        ### YOUR SOLUTION HERE
        
        
    return maha_dist

maha_dist = maha_dist_generator(centered_data)
cluster_contour_plot(centered_data, 3, maha_dist, "Mahalanobis distance", 1.05)

#### From these 4 distance metrics, which do you think would be most suitable to use as a distance between the petal length and petal width of the 150 flowers in this data set? Explain your answer.

*Your answer goes here.*

#### Does the shape of the clusters, when using the distance metric you answered in the previous question, correspond with what you would expect when looking at the data? Explain your answer.

*Your answer goes here.*


# Elbow method

Now that you have a working k-means implemenation and have selected your prefered distance metric, we can answer the last remaining question in this assignment:

* What is the most likely number of cluster $k$ for this data set?

One way to answer this question, is using the elbow method, as described in the theory videos. Complete the following 3 functions below to create an elbow graph:

1. `cost`: This should *not necessarily* be the same cost function as defined in the theory videos, as the cost of a set of fitted cluster should depend on the way the distance between the mean and the points in that cluster is defined. Write a sensible cost that should be minimized with each step of the k-means algorithm. You may write a function specifically for the distance metric you choose, but ideally the cost should work for *any* definition of distance.
2. `average_cost`: This function should repeatedly try and fit `k` clusters to the data, and compute the resulting convered cost. As this cost will depend on the cluster initialization, the process should be repeated `n` times and the cost averaged.
3. `create_elbow_plot`: This function should compute the average cost from $1$ to `max_k`, using the provided `data` and `dist_func`, and create the corresponding elbow plot.

Finally, call `create_elbow_plot` with a `max_k` of $10$ and using your preferred distance metric.

In [ ]:
def cost(data, means, b, dist_func):
    ### YOUR SOLUTION HERE
    

def average_cost(data, k, dist_func, n):
    ### YOUR SOLUTION HERE
    

def create_elbow_plot(data, max_k, dist_func, n=10):
    ### YOUR SOLUTION HERE
    
    
### YOUR SOLUTION HERE


#### What do you think is the most likely value for $k$ in the Iris data set, using your preferred distance metric? Explain your answer.

*Your answer goes here.*
